In [2]:
from transformers import MT5Model
from transformers import AdamW
from transformers import MT5Tokenizer
import pickle

In [3]:
model = MT5Model.from_pretrained("google/mt5-small")
model.train()

tokenizer = MT5Tokenizer.from_pretrained('google/mt5-small')
optimizer = AdamW(model.parameters(), lr=0.00001)

print("Number of parameters: ", model.num_parameters())

Some weights of the model checkpoint at google/mt5-small were not used when initializing MT5Model: ['lm_head.weight']
- This IS expected if you are initializing MT5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MT5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Number of parameters:  172119424


In [4]:
tokenizer = MT5Tokenizer.from_pretrained('google/mt5-small')
optimizer = AdamW(model.parameters(), lr=0.00001)

In [5]:
with open("rusDialogs/prepared_dialogueCR.pickle", 'rb') as file:
    dialogues = pickle.load(file)

## even stands for even number of repliques in one dialog, so each U1 replique has an answer
dialogues_even = list(map(lambda x: x[:-1] if len(x) % 2 == 1 else x, dialogues))
dialogues_even_flatten = [replique for dialogue in dialogues for replique in dialogue]
repliquesU1 = list(filter(lambda x: x.startswith("USER1:"), dialogues_even_flatten))
repliquesU2 = list(filter(lambda x: x.startswith("USER2:"), dialogues_even_flatten))

In [6]:
# encoding = tokenizer(text_batch, return_tensors='pt', padding=True, truncation=True)
# input_ids = encoding['input_ids']
# attention_mask = encoding['attention_mask']

In [7]:
batch = tokenizer.prepare_seq2seq_batch(src_texts=repliquesU1, tgt_texts=repliquesU2, return_tensors="pt", padding=True)

In [8]:
outputs = model(input_ids=batch.input_ids[:3], decoder_input_ids=batch.labels[:3])

In [9]:
outputs.last_hidden_state.shape

torch.Size([3, 430, 512])

In [10]:
model.generate(repliquesU1[0], max_length=250, do_sample=True, top_p=0.95, top_k=60)

TypeError: 'in <string>' requires string as left operand, not int